## **Nugen Intelligence**
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="200"/>

Domain-aligned foundational models at industry leading speeds and zero-data retention! To learn more, visit [Nugen](https://docs.nugen.in/introduction)

### **Using Reasoning for Routine Generation with Nugen API**
 
This cookbook will guide you through using the Nugen API to generate routines based on customer service policies. We will explain each step of the code in simple terms, so even if you're new to programming or APIs, you'll be able to follow along. In this cookbook, we use Python and the Nugen API to convert help center articles into internal-facing, executable routines.

The goal is to enhance customer service operations by enabling the LLM to effectively handle customer inquiries and support tasks, ensuring a more efficient and responsive service experience. With Nugen’s advanced API, you can streamline your customer service processes and empower your team with actionable insights from existing help center content.

**Importing Necessary Libraries**


In [ ]:
!pip install --quiet pandas requests

import requests
from IPython.display import display, HTML
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import csv

* requests: A library to make HTTP
* requests to the Nugen API.
* pandas: A library to handle data manipulation and analysis.
* ThreadPoolExecutor: A tool to run multiple tasks concurrently, speeding up processing.
* csv: A module to read and write CSV files.

**Step 2: Set up the Nugen API Client**

To read more about Nugen API and access free API keys, you can visit [Nugen Dashboard](https://nugen-platform-frontend.azurewebsites.net/dashboard)

In [ ]:
url_api_server = "https://api.nugen.in"
api_key =  <enter your api key> # Replace with your actual Nugen API key
MODEL = "nugen-flash-instruct"

Here, we define the API base URL and your API key. Replace <--nugen api key--> with your actual key to authenticate your requests to the Nugen API. The MODEL variable specifies the model we will use for generating the routines.

**Step 3: Create the NugenAPIClient Class**

In [ ]:
# Nugen API Client Class
class NugenAPIClient:
    def __init__(self, base_url, api_key):
        self.base_url = base_url
        self.api_key = api_key

    def chat_completions_create(self, model, messages, max_tokens=400, temperature=1):
        """
        Makes a call to Nugen API to create chat completions.
        """
        url = f"{self.base_url}/inference/completions"
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        prompt = "\n".join([message["content"] for message in messages])

        payload = {
            "model": model,
            "prompt": prompt,
            "max_tokens": max_tokens,
            "temperature": temperature
        }

        response = requests.post(url, json=payload, headers=headers)

        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error {response.status_code}: {response.text}")

This class helps us interact with the Nugen API. It stores the base_url and api_key so they can be used in every request without repeating the values.

1. This method takes the model, messages, and other parameters to generate a response from the API.

2. We use the requests.post method to send the data (payload) to the API and retrieve the result.

3. If the request is successful (status_code == 200), we return the response. If not, an error is raised.

**Step 5: Initialize the Client**

In [12]:
# Instantiate Nugen client
client = NugenAPIClient(base_url=url_api_server, api_key=api_key)

Here, we create an instance of NugenAPIClient using the API server URL and your API key. This instance will be used to make requests to the API.

**Step 6: Prepare the Conversion Prompt**

In [ ]:
# Conversion Prompt for Chain of Thought (CoT) Reasoning
CONVERSION_PROMPT = """
You are a helpful assistant tasked with taking an external facing help center article 
and converting it into an internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions 
from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the customer service policy carefully** to ensure every step is accounted for. 
   It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format.
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b). Sub-actions start 
     on new lines.
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was 
     purchased within 30 days, then...').
   - **Actions requiring external systems** should call a function (e.g., `call check_delivery_date`).
   - **Prioritize compliance** by adhering to privacy regulations, policies, and legal requirements.
   - **Handle exceptions** by specifying steps for scenarios outside standard policies.

**Important**: If uncertain, respond with "I don't know."
"""

# Routine Quality Assurance Validator Prompt
QA_VALIDATOR_PROMPT = """
You are a validator LLM. Your job is to assess the generated routine for:

1. Completeness: Does the routine cover all aspects of the original policy?
2. Legal and Compliance Adherence: Check against HIPAA, GDPR, PCI DSS, or other relevant policies.
3. Accuracy: Does the routine faithfully represent the original policy?
4. Clarity: Ensure the routine is easy to understand and executable programmatically.

Return either "PASS" or "FAIL" with a short reason explaining your evaluation.
"""

### Conversion Prompt for Chain of Thought (CoT) Reasoning

The `CONVERSION_PROMPT` guides an LLM to convert help center articles into internal-facing, executable routines optimized for programmatic use. The routine:
- Ensures careful review of all customer service policies, maintaining completeness.
- Follows a logical, numbered structure with sub-actions for clarity and step-by-step execution.
- Uses conditionals (`if...then...else`) and specifies actions requiring external systems, while adhering to compliance requirements.
- Encourages accurate translation of policies, with a fallback response of "I don't know" for uncertain scenarios.

---

### Routine Quality Assurance Validator Prompt

The `QA_VALIDATOR_PROMPT` assesses generated routines based on the following:
1. **Completeness**: Ensures all aspects of the original policy are covered.
2. **Legal and Compliance Adherence**: Verifies compliance with HIPAA, GDPR, PCI DSS, or similar regulations.
3. **Accuracy**: Confirms the routine faithfully represents the original policy.
4. **Clarity**: Checks for ease of understanding and programmatic execution.

The validator returns either a "PASS" or "FAIL" with a brief explanation, ensuring high standards for generated routines.

This prompt instructs the API on how to convert the policy text into a routine. The instructions are detailed to ensure the generated routine is accurate and executable.

**Step 7: Read Policies from a CSV File**

In [14]:
articles = []
with open('../helpcenter_articles.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        articles.append({
            "policy": row["policy"],
            "content": row["content"]
        })

This section reads the help center articles from a CSV file and stores each article’s policy and content in a list called articles.

**Step 8: Generate Routine Using the Nugen API**

**Processing Articles Concurrently**


In [15]:
def generate_routine(policy):
    try:
        messages = [{
            "role": "user",
            "content": f"{CONVERSION_PROMPT}\n\nPOLICY:\n{policy}"
        }]

        response = client.chat_completions_create(
            model=MODEL,
            messages=messages
        )

        # Print the full response to debug
        print("Full API Response:", response)

        # Modify this according to the actual response structure
        return response['choices'][0].get('text', None)  # Adjust based on actual API response
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

1. This function sends a policy to the Nugen API and requests a converted routine.

2. It prints the full API response for debugging and returns the generated routine text if available.

In [ ]:
# Routine Quality Assurance Validator Function
def evaluate_routine_for_quality(routine, policy_text):
    """
    Evaluates the generated routine for legal compliance, accuracy, and clarity using a validator.
    """
    try:
        messages = [{
            "role": "user",
            "content": f"{QA_VALIDATOR_PROMPT}\n\nPolicy:\n{policy_text}\n\nRoutine:\n{routine}"
        }]
        review = client.chat_completions_create(model=MODEL, messages=messages)
        evaluation = review['choices'][0].get('text', '').lower()
        if "pass" in evaluation:
            return True
        else:
            print(f"Routine failed validation: {evaluation}")
            return False
    except Exception as e:
        print(f"An error occurred during validation: {e}")
        return False

### Routine Quality Assurance Validator Function

The `evaluate_routine_for_quality` function evaluates a generated routine against a provided policy using a validator LLM. 

#### **Steps Explained**:
1. **Purpose**:
   - Validates the routine for legal compliance, accuracy, and clarity, ensuring it aligns with predefined standards.

2. **Prompt Composition**:
   - Combines the `QA_VALIDATOR_PROMPT`, `policy_text`, and `routine` into a structured input for the validator LLM.

3. **API Call**:
   - Sends the prompt to the Nugen API using the `client.chat_completions_create` method to retrieve the evaluation result.

4. **Evaluation Logic**:
   - Parses the API response and checks if it contains "pass" to indicate successful validation. If "pass" is not found, the failure reason is printed.

5. **Error Handling**:
   - Gracefully manages exceptions, such as API call failures, ensuring the program does not crash.

#### **Output**:
- Returns `True` for a "PASS".
- Prints the failure reason and returns `False` if validation fails.
- Handles errors with informative messages for debugging.

This function streamlines the validation process, combining automation and robust handling for efficient routine quality assurance.

**Step 9: Process Each Article**

In [ ]:
# Article Processing Function (with validation)
def process_article(article):
    routine = generate_routine(article['content'])
    if routine:
        # Validate the routine for accuracy, legal compliance, and clarity
        if evaluate_routine_for_quality(routine, article['content']):
            validation_status = "PASS"
        else:
            validation_status = "FAIL"
    else:
        validation_status = "FAIL"
    
    return {"policy": article['policy'], "content": article['content'], "routine": routine, "validation_status": validation_status}


### Article Processing Function with Validation

The `process_article` function processes an article by generating a routine and validating it for accuracy, compliance, and clarity.

#### **Steps**:
1. **Routine Generation**:
   - Calls the `generate_routine` function to create a routine based on the article's content.
   
2. **Validation**:
   - If the routine is successfully generated, it passes to the `evaluate_routine_for_quality` function for validation.
   - The routine is checked against the article's content, ensuring adherence to policy standards.
   - Validation outcomes:
     - **"PASS"**: Routine meets standards for quality.
     - **"FAIL"**: Routine fails validation or was not generated.

3. **Output**:
   - Returns a dictionary containing:
     - The article's policy.
     - The article's content.
     - The generated routine.
     - The validation status (either "PASS" or "FAIL").

This function efficiently combines generation and validation, ensuring the resulting routines are robust and compliant.

**Step 10: Execute Concurrently for Efficiency**

Here, we use ThreadPoolExecutor to process all articles concurrently. This makes the code run faster when dealing with multiple articles.

In [ ]:
# Process Articles in Parallel
with ThreadPoolExecutor() as executor:
    results = list(executor.map(process_article, articles))

**Step 11: Convert Results into a DataFrame**

In [ ]:
df = pd.DataFrame(results)

We store the processed results in a pandas DataFrame for easy manipulation and display.

**Step 12: Display the Data in a User-Friendly Way**

In [ ]:
# Display Formatted DataFrame Function
def display_formatted_dataframe(df):
    def format_text(text):
        return text.replace('\n', '<br>') if text else "No routine generated"

    df_formatted = df.copy()
    df_formatted['content'] = df_formatted['content'].apply(format_text)
    df_formatted['routine'] = df_formatted['routine'].apply(format_text)

    display(HTML(df_formatted.to_html(escape=False, justify='left')))

1. This function formats the DataFrame, replacing newlines with HTML line breaks for better readability.

2. The final DataFrame is displayed using IPython.display with proper formatting to make it easier to read in a notebook environment.

**Summary**

By following these steps, you can interact with the Nugen API, process help center articles, and generate routines for internal use. This guide is beginner-friendly and can be adapted to various use cases.